In [8]:
import os
import numpy as np

from Stationary.core import *

from sklearn import linear_model, datasets
from sklearn.metrics import mean_squared_error, r2_score

import matplotlib.pyplot as plt


In [9]:
w1 = Worker(coef = np.array([0, 0]), intercept = np.array([0]), model=linear_model.LinearRegression())
w2 = Worker(coef = np.array([0, 0]), intercept = np.array([0]), model=linear_model.LinearRegression())
w3 = Worker(coef = np.array([0, 0]), intercept = np.array([0]), model=linear_model.LinearRegression())

In [10]:
l = []
l.append(w1)
l.append(w2)
l.append(w3)

In [11]:
dataset_X, dataset_y = datasets.load_diabetes(return_X_y=True, )

In [12]:
server = Server(coef = np.array([0, 0]), intercept = np.array([0]), workers = l, num_features = dataset_X[0].size)

In [13]:
train_w1_X, train_w2_X, train_w3_X, test_X = np.split(dataset_X, 
                                                      [int(.25 * len(dataset_X)), 
                                                       int(.50 * len(dataset_X)), 
                                                       int(.75 * len(dataset_X))
                                                      ])

In [14]:
train_w1_y, train_w2_y, train_w3_y, test_y = np.split(dataset_y, 
                                                      [int(.25 * len(dataset_y)), 
                                                       int(.50 * len(dataset_y)), 
                                                       int(.75 * len(dataset_y))
                                                      ])

In [15]:
train_w1_1, train_w1_2, train_w1_3 = np.split(train_w1_X,  
                                                [int(.33 * len(train_w1_X)), 
                                                int(.66 * len(train_w1_X))
                                                ])

train_w2_1, train_w2_2, train_w2_3 = np.split(train_w2_X,  
                                                [int(.33 * len(train_w2_X)), 
                                                int(.66 * len(train_w2_X))
                                                ])

train_w3_1, train_w3_2, train_w3_3 = np.split(train_w3_X,  
                                                [int(.33 * len(train_w3_X)), 
                                                int(.66 * len(train_w3_X))
                                                ])




train_w1_y_1, train_w1_y_2, train_w1_y_3 = np.split(train_w1_y, 
                                                      [int(.33 * len(train_w1_y)), 
                                                       int(.66 * len(train_w1_y))
                                                      ])

train_w2_y_1, train_w2_y_2, train_w2_y_3 = np.split(train_w2_y, 
                                                      [int(.33 * len(train_w2_y)), 
                                                       int(.66 * len(train_w2_y))
                                                      ])

train_w3_y_1, train_w3_y_2, train_w3_y_3 = np.split(train_w3_y, 
                                                      [int(.33 * len(train_w3_y)), 
                                                       int(.66 * len(train_w3_y))
                                                      ])

In [8]:
w1.train(X = train_w1_X, y = train_w1_y)
w2.train(X = train_w2_X, y = train_w2_y)
w3.train(X = train_w3_X, y = train_w3_y)


In [9]:
w3.model.coef_

array([  57.18276806,  -75.1429839 ,  537.2001464 ,  370.81987107,
         43.25106484,   37.51865177, -331.11325767, -154.85685861,
        542.60138261,  -31.62912061])

In [10]:
server.aggregation()

In [11]:
print('Server Coef: ', server.coef)
print('Server intercept: ', server.intercept)

Server Coef:  [  -4.08894884 -202.30961161  511.95852512  280.21844215 -322.12129046
   70.23540153  -41.16479233  135.70885414  625.80283674   74.97544225]
Server intercept:  [150.45345618]


In [12]:
server.return_to_workers()

In [13]:
w3.coef

array([  -4.08894884, -202.30961161,  511.95852512,  280.21844215,
       -322.12129046,   70.23540153,  -41.16479233,  135.70885414,
        625.80283674,   74.97544225])

In [14]:
pred_w1 = w1.evaluate(test_X)
print('Mean squared error worker 1: %.2f' 
         % mean_squared_error(test_y, pred_w1))
print('Coefficient of determinaton of worker 1: %.2f'
         % r2_score(test_y, pred_w1))

print('----------')

pred_w2 = w2.evaluate(test_X)
print('Mean squared error worker 2: %.2f' 
         % mean_squared_error(test_y, pred_w2))
print('Coefficient of determinaton of worker 2: %.2f'
         % r2_score(test_y, pred_w2))

print('----------')

pred_w3 = w3.evaluate(test_X)
print('Mean squared error worker 3: %.2f' 
         % mean_squared_error(test_y, pred_w3))
print('Coefficient of determinaton of worker 3: %.2f'
         % r2_score(test_y, pred_w3))

Mean squared error worker 1: 3226.93
Coefficient of determinaton of worker 1: 0.47
----------
Mean squared error worker 2: 2978.78
Coefficient of determinaton of worker 2: 0.51
----------
Mean squared error worker 3: 2893.24
Coefficient of determinaton of worker 3: 0.53


In [15]:
server.return_to_workers()

In [16]:
pred_server = server.evaluate(test_X)
print('Mean squared error worker 1: %.2f' 
         % mean_squared_error(test_y, pred_server))
print('Coefficient of determinaton of worker 1: %.2f'
         % r2_score(test_y, pred_server))

Mean squared error worker 1: 2811.18
Coefficient of determinaton of worker 1: 0.54


In [18]:
pred_server = server.evaluate(test_X)
print('Mean squared error worker 1: %.2f' 
         % mean_squared_error(test_y, pred_server))
print('Coefficient of determinaton of worker 1: %.2f'
         % r2_score(test_y, pred_server))

Mean squared error worker 1: 2811.18
Coefficient of determinaton of worker 1: 0.54


# Multi Rounds

In [ ]:
w1.train(X = train_w1_1, y = train_w1_y_1)
w2.train(X = train_w2_1, y = train_w2_y_1)
w3.train(X = train_w3_1, y = train_w3_y_1)

server.aggregation()
server.return_to_workers()
